My room bookcase
=============

# Klimpen bookcase
http://www.ikea.com/it/it/catalog/products/10353765/

![klimpen full size](./images/klimpen_1.png)
![klimpen detail](./images/klimpen_2.png)


Note: measures are in meters

In [1]:
from larlib import *

Evaluating fenvs.py..
...fenvs.py imported in 0.012899 seconds


# Bookcase structure
We will build the various part of the bookcase, then we will put them together.
A second approach will be shown after (PyPlasm $Q(arg)$ function will be used).

Structure:
- back: as suggested is the back side of our bookcase, it will be attached to the shelves, the side block, the top and the bottom.
- shelves
- top
- bottom

Kplimpen is offered as a 50x70x33 double shelves bookcase. For this woorkshoop we will handle a parametrized (in respect of the dimension and color) Klimpen

## Back of the bookcase

In [9]:
def back(width, height):
    """
    back: builds the back of the bookcase
    :param back: height, width in meters
          thickness is fixed
    :return: a CUBOID. The 3d model for the back
    """ 
    return CUBOID([width,height,0.1]) #0.1 is thickness

In [8]:
b = back(5.8, 7.)
VIEW(b)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x1812221150> >

## Shelves

In [13]:
def shelf(width, height, length):
    """
    shelf: builds a single shelf
    :param width: width of the shelf
    :return: a CUBOID. The 3D model for a shelf
    """
    return CUBOID([width,height,length])


def shelves(nshelves, heightBack, width, height, length):
    """
    shelves builds <nshelves> shelves
    :param nshelves: number of shelves
    :param heightBack: height of the back, the area where shelves are positionable
    :param width: width of the shelf.
    :param height: height of the shelf.
    :param length: length of the shelf. (z-axis)
    :return: nshelves CUBOIDs. nshelves shelves
    """
    interval = heightBack/(nshelves+1)
    s = STRUCT(NN(nshelves) ([shelf(width, height, length), T(2)(interval)]))
    return s

In [26]:
ss = shelves(nshelves=2, heightBack=5.8, width=6.8, height=0.2, length=3.0)
VIEW(ss)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181222b690> >

![2 shelves](./images/2_shelves.png)

## Sides
The lateral components. Two parallel wooden beam

In [34]:
def side(heigth, length):
    """
    side: side of the bookcase structure
    :param heigth: height of the wooden beam
    :param length: length of the wooden beam
    :return: a CUBOID. The 3d model for a lateral wooden beam
    """
    return CUBOID([0.3, heigth, length])

In [37]:
def sides(height, length, distance):
    """
    sides: a couple of sides
    :param height: as side(height,length)
    :param length:  "
    :param distance:  "
    :return: STRUCT([CUBOID,CUBOID])
    """
    s = side(height, length)
    sides = STRUCT([s, T(1)(distance)(s)])
    return sides

In [31]:
single_side = side(5.8, 3.0)
VIEW(single_side)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x181f1a0060> >

In [ ]:
two_sides = sides(5.8, 3.0, 7.0)
VIEW(two_sides)

The lateral components of our bookcase

![sides](./images/sides.png)

## Top
The top of our bookcase. It consists of a set of CUBOIDs placed one on top of another

In [ ]:
def top(widthBack, height, lengthBack):
    """
    top: the top of the bookcase
    :param widthBack: the width of the back side of the bookcase will be used to compute the 
    :param height: the height
    :param lengthBack: the length
    :return: a STRUCT made of CUBOIDs (layer_i).
    """
    
    layer1 = CUBOID([width, height, length])
    return 